In [11]:
from typing import Union
import google.generativeai as genai
import os
from dotenv import load_dotenv, find_dotenv


In [14]:
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b

In [15]:
def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Subtracts two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.
    """
    return a - b


In [16]:
def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Multiplies two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Product of the two numbers.
    """
    return a * b




In [29]:
import os
import logging
from dotenv import load_dotenv, find_dotenv
import google.generativeai as genai

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
if not load_dotenv(find_dotenv()):
    logging.warning("Could not find .env file. Make sure it exists and contains the required API key.")

# Fetch API key
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise ValueError("Missing GOOGLE_API_KEY in environment variables.")

# Configure GenAI
genai.configure(api_key=API_KEY)

def get_completion(system_prompt: str, user_prompt: str, model: str = "gemini-2.0-flash") -> str:
    """
    Generates a response using Google's Gemini API.

    Args:
        system_prompt (str): The system prompt providing context.
        user_prompt (str): The user's input prompt.
        model (str): The AI model to use (default: gemini-2.0-flash).

    Returns:
        str: The AI-generated response.
    """
    try:
        model_instance = genai.GenerativeModel(model)
        full_prompt = f"{system_prompt}\n\n{user_prompt}"
        response = model_instance.generate_content(full_prompt)

        if not response or not hasattr(response, "text"):
            raise ValueError("Invalid response format from API.")

        return response.text.strip()
    
    except Exception as e:
        logging.error(f"Error generating response: {e}")
        return "An error occurred while generating the response."

def main():
    system_prompt = ''' You are an AI assistant who solves mathematical problems. You will be given Python 
    functions to use for problem-solving. Your response should return only the function definition (without a call)
    so that it can be executed using Python's built-in exec().

```
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b
```

```
def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Subtracts two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.
    """
    return a - b
```

```
def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Multiplies two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Product of the two numbers.
    """
    return a * b
```

'''

    user_prompt = "add two numbers 10 and 20"
    
    response = get_completion(system_prompt, user_prompt)
    print(response)

    exec(response)

if __name__ == "__main__":
    main()


```python
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """
    Adds two numbers and returns the result.

    Args:
        a (Union[int, float]): First number.
        b (Union[int, float]): Second number.

    Returns:
        Union[int, float]: Sum of the two numbers.
    """
    return a + b
```


SyntaxError: invalid syntax (<string>, line 1)

In [33]:
from typing import Any, Callable, Dict, List, Optional, Union
import inspect
import json
import logging
import re
from dataclasses import dataclass
from enum import Enum

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FunctionType(Enum):
    MATH = "math"
    UTILITY = "utility"
    SYSTEM = "system"

@dataclass
class FunctionMetadata:
    name: str
    description: str
    parameters: Dict[str, Dict[str, Any]]
    returns: Dict[str, Any]
    type: FunctionType

class FunctionRegistry:
    def __init__(self):
        self._functions: Dict[str, Callable] = {}
        self._metadata: Dict[str, FunctionMetadata] = {}
    
    def register(self, func_type: FunctionType):
        """Decorator to register functions with metadata"""
        def decorator(func: Callable):
            sig = inspect.signature(func)
            doc = inspect.getdoc(func) or ""
            
            # Extract parameter information
            parameters = {}
            for name, param in sig.parameters.items():
                param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "Any"
                parameters[name] = {
                    "type": param_type,
                    "description": "",
                    "required": param.default == inspect.Parameter.empty
                }
            
            # Create function metadata
            metadata = FunctionMetadata(
                name=func.__name__,
                description=doc.split('\n')[0],
                parameters=parameters,
                returns={"type": sig.return_annotation.__name__ if sig.return_annotation != inspect.Signature.empty else "Any"},
                type=func_type
            )
            
            self._functions[func.__name__] = func
            self._metadata[func.__name__] = metadata
            return func
        return decorator
    
    def get_function(self, name: str) -> Optional[Callable]:
        return self._functions.get(name)
    
    def get_metadata(self, name: str) -> Optional[FunctionMetadata]:
        return self._metadata.get(name)
    
    def list_functions(self, func_type: Optional[FunctionType] = None) -> List[str]:
        if func_type:
            return [name for name, meta in self._metadata.items() if meta.type == func_type]
        return list(self._functions.keys())

class AIFunctionAgent:
    def __init__(self, model_client: Any, registry: FunctionRegistry):
        self.model = model_client
        self.registry = registry
    
    def _parse_response(self, response_text: str) -> Dict[str, Any]:
        """
        Parse the model's response into a structured format.
        Handles both JSON and non-JSON responses.
        """
        # First try parsing as JSON
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            pass
        
        # If not JSON, try to extract function name and parameters
        try:
            # Look for function name pattern
            func_match = re.search(r'(add|sub|mul)_two_numbers', response_text)
            if not func_match:
                raise ValueError("No valid function found in response")
            
            func_name = func_match.group(0)
            
            # Look for numbers in the text
            numbers = re.findall(r'\d+(?:\.\d+)?', response_text)
            if len(numbers) < 2:
                raise ValueError("Could not find two numbers in response")
            
            # Convert to appropriate type (int or float)
            params = [
                float(num) if '.' in num else int(num)
                for num in numbers[:2]
            ]
            
            return {
                "function": func_name,
                "parameters": {
                    "a": params[0],
                    "b": params[1]
                }
            }
        except Exception as e:
            logger.error(f"Error parsing non-JSON response: {e}")
            raise ValueError(f"Could not parse response: {response_text}")
    
    def _create_system_prompt(self) -> str:
        """Creates a system prompt describing available functions"""
        function_descriptions = []
        for name in self.registry.list_functions():
            metadata = self.registry.get_metadata(name)
            if metadata:
                function_descriptions.append(
                    f"Function: {name}\n"
                    f"Description: {metadata.description}\n"
                    f"Parameters: {', '.join(metadata.parameters.keys())}\n"
                )
        
        return f"""You are an AI assistant that helps execute mathematical functions. 
Available functions:

{'\n'.join(function_descriptions)}

When responding, either:
1. Return a JSON object in this format:
{{
    "function": "function_name",
    "parameters": {{
        "a": first_number,
        "b": second_number
    }}
}}

2. Or simply state the function name and parameters clearly in plain text.
For example: "Use add_two_numbers with a=10 and b=20"
"""

    def execute(self, user_prompt: str) -> Any:
        """Processes user prompt and executes the appropriate function"""
        try:
            # Get model's function selection
            system_prompt = self._create_system_prompt()
            response = self.model.generate_content(f"{system_prompt}\n\nUser request: {user_prompt}")
            
            logger.info(f"Model response: {response.text}")
            
            # Parse response
            function_call = self._parse_response(response.text)
            
            # Validate function exists
            func_name = function_call["function"]
            function = self.registry.get_function(func_name)
            if not function:
                raise ValueError(f"Function '{func_name}' not found")
            
            # Execute function
            result = function(**function_call["parameters"])
            return result
            
        except Exception as e:
            logger.error(f"Error executing function: {e}")
            raise

# Example usage
registry = FunctionRegistry()

@registry.register(FunctionType.MATH)
def add_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Adds two numbers and returns the result."""
    return a + b

@registry.register(FunctionType.MATH)
def sub_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Subtracts two numbers and returns the result."""
    return a - b

@registry.register(FunctionType.MATH)
def mul_two_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Multiplies two numbers and returns the result."""
    return a * b

def main():
    # Initialize the model client
    model = genai.GenerativeModel("gemini-2.0-flash")
    
    # Create the agent
    agent = AIFunctionAgent(model, registry)
    
    # Execute a function based on user prompt
    try:
        result = agent.execute("substract two numbers 10 and 20")
        print(f"Result: {result}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

2025-02-10 16:59:47,509 - INFO - Model response: Use sub_two_numbers with a=10 and b=20



Result: -10


this is fantastic code. Works really great. 
However, for me it is very difficult to understand. And I would like to extend this functionality. I may add many more functions with verity of input and outputs. 

can you make it simpler?